In [ ]:
# written by Maximilian Ullherr, maximilian.ullherr@physik.uni-wuerzburg.de,
# Lehrstuhl fuer Roentgenmikroskopie/Universitaet Wuerzburg, Josef-Martin-Weg 63, 97074 Wuerzburg, Germany
# last updated 2019-08-01

%pylab
import sys
from ball_phantom_evaluation import *
from notebook import *

print('\n=== printing the docstring for the main function+class ===')
help(simulate_ball_phantom)
help(BallVolumeEvaluator)

In [ ]:
result_cache.enable_cache('cache')

In [ ]:
result_cache.clear_cache()

# Simulation

In [ ]:
print('generating a random ball volume')
size = 500 # runtime should scale with size**6*voxelsize**3
size = 300
voxelsize = 0.5
ball_sizes = (10, 20.3)
ball_sizes = (10, 20.3, 28.1) # 2 ball populations are sufficient, 3 are used for testing purposes

cache_data = result_cache.Data('ball_volume_mask', size, voxelsize, ball_sizes)
try:
    ideal_phantom, centers, diameters = cache_data.load()
except result_cache.NoData:
    ideal_phantom, centers, diameters = simulate_ball_phantom((size, size, size), ball_sizes, 
                                                          (8, 4, 1), voxelsize, size_error=0.03)
    cache_data.save(ideal_phantom, centers, diameters)

print(f'number of simulated balls: {len(diameters)}')

fig = show_images([ideal_phantom[:, 2, :], ideal_phantom[ideal_phantom.shape[0]//2]], ["upper slice", "center slice"])
display(fig)
plt.close(fig)
print(ideal_phantom.shape)

In [ ]:
print('simulating a phase contrast measurement')
p, sigma, mu, SNR_fact = 60, 1.7, 0.7, 2

cache_data = result_cache.Data('ball_volume_phase', ideal_phantom, p, sigma, mu, SNR_fact)
try:
    data_phantom, data_noise = cache_data.load()
except result_cache.NoData:
    FF = image.FourierFilterer(shape=ideal_phantom.shape)
    u = FF.u_radial
    MTF_sim = (1 + p**2*u**2)*image.MTF_gauss(u, sigma)*image.MTF_exp(u, mu)
    data_phantom = FF.apply_filter(ideal_phantom, MTF_sim)
    data_noise = np.random.randn(*data_phantom.shape)*np.std(data_phantom)/SNR_fact
    data_phantom += data_noise

    cache_data.save(data_phantom, data_noise)
    
z = ideal_phantom.shape[0]//2
display(show_images([ideal_phantom[z], data_phantom[z]]))

In [ ]:
morphology.verbose = True

# note that the segmented data (first argument) is here chosen from the ideal phantom
# WARNING: data_is_volume_simulation=True argument must not be used on real data

BVE_1 = BallVolumeEvaluator(ideal_phantom > (OS_factor**3//2), data_phantom, ball_sizes, voxelsize=voxelsize, 
                            size_error_fraction=0.1, data_is_volume_simulation=True, 
                            data_noise=data_noise[20:], save_path='test/ball_phantom')
BVE_1.log += 'this is a test run of the BallVolumeEvaluator on simulated data\n'  # will be saved to ... log.txt

BVE_1.evaluate()
display(BVE_1.images_fig)
display(BVE_1.hist_fig)
print('note on the model fit: in real applications, fixed parameters and well chosen starting parameters may be needed')
display(BVE_1.model_fit_fig)
display(BVE_1.power_spectra_fig)
plt.close('all')

In [ ]:
# if the fit fails (it works on the simulated data):
# repeat fit with different starting values (p0=) and fixed parameters (fixed=)
# see the definition of ball_phantom_evaluation.ModelMTF.__call__() for the meaning of the parameters
BVE_1.fit_model(p0=(1, 60, 1, 1, 1, 1, 1, 1), fixed=dict(c=0, mu_n=0, sigma_n=0))  # c=0, ... is white noise
display(BVE_1.model_fit_fig)
display(BVE_1.power_spectra_fig)
BVE_1.save_results()  # comment this out when you experiment with fit parameters
plt.close('all')